In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
KEY=os.getenv('api_key')
# print(KEY)

In [4]:
llm=ChatOpenAI(
    model="mistralai/mistral-7b-instruct:free",
    openai_api_key=KEY,  # your OpenRouter key
    base_url="https://openrouter.ai/api/v1",
    temperature=0.5,)


C:\Users\Dell\AppData\Local\Temp\ipykernel_8712\2929340663.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm=ChatOpenAI(


In [5]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000024D40EC2A60>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000024D40EE8280>, model_name='mistralai/mistral-7b-instruct:free', temperature=0.5, openai_api_key='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', openai_api_base='https://openrouter.ai/api/v1', openai_proxy='')

In [6]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ Maker. Given a above text, it is you job to \
create the quiz od {number} multiple choice questions for {subject} student in {tone} tone.
Make sure the questions are not repeated and check all the question to be confirming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it  as a guide. \
Ensure to make the {number} MCQ's
### RESPONSE_JSON
{response_json}


"""

In [9]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [10]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_8712\3899119271.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)


In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(input_variables=['subject','quiz'],template=TEMPLATE)

In [13]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [14]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],output_variables=[
"quiz","review"],verbose=True)

In [17]:
generate_evaluate_chain



SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template="\nText:{text}\nYou are an expert MCQ Maker. Given a above text, it is you job to create the quiz od {number} multiple choice questions for {subject} student in {tone} tone.\nMake sure the questions are not repeated and check all the question to be confirming the text as well.\nMake sure to format your response like RESPONSE_JSON below and use it  as a guide. Ensure to make the {number} MCQ's\n### RESPONSE_JSON\n{response_json}\n\n\n"), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000024D40EC2A60>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000024D40EE8280>, model_name='mistralai/mistral-7b-instruct:free', temperature=0.5, openai_api_key='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', openai_api_base='

In [18]:
file_path=r"C:\Users\Dell\mcqgenai\data.txt"
file_path

'C:\\Users\\Dell\\mcqgenai\\data.txt'

In [19]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [20]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cognitive systems contribu

In [21]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [22]:

NUMBER=5
SUBJECT="data science"
TONE="simple"

In [23]:
response = generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone":TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)


C:\Users\Dell\AppData\Local\Temp\ipykernel_8712\3057265675.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied h

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers

In [26]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]\n\nThe earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cognitive sys

In [67]:
quiz=response.get("quiz")

In [68]:
quiz_json = quiz.split("### RESPONSE_JSON")[-1].strip()
# quiz.split("### RESPONSE_JSON")[-1].strip()
quiz=json.loads(quiz_json)

In [59]:
quiz_table_data=[]
for key,val in quiz.items():
    mcq=val["mcq"]
    options=" | ".join([
        f"{option} : {option_value}"
        for option,option_value in val["options"].items()
    ])
    correct=val["correct"]
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Correct":correct})


In [71]:
quiz=pd.DataFrame(quiz_table_data)

In [72]:
quiz.to_csv("machine_learning.csv")